# Some Details on Building Equity Index Option Analytics

## The Problem

Build a risk graph and compute greeks for a portfolio of index options.

- Not covering options on ETF, futures or single stocks.
- Focus on what can be extracted from a single snapshot of an option chain
  - All expirations, strikes, calls and puts have bid/ask sampled at the same time.
  - Not assuming spot index or futures quotes available.
  - Not assuming interest rates or dividend yields are available.

## Procedure Outline

1. Start with a table of option prices:  
Expiration, Strike, PutOrCall, Bid, Ask
2. For each expiration, assign a `dte` and set `YearFrac` $\tau = \text{dte}/365.$
3. For each expiration, assign an interest rate, $r$, and discount factor:
  $D = \exp(-r\tau)$.
4. For each expiration, assign a forward index price, $F$.  
  Assign a spot price, $S$, for all expirations, and 
  a dividend yield, $\delta$, for each expiration and use
  $F = S \exp( (r-\delta)\tau)$.
5. For each call and put option, compute a mark between the bid and ask prices.
6. Compute an implied volatility for each call and put option.
7. Compute greeks for each call and put option.  Aggregate to get portfolio greeks.
8. Build a "sticky strike" risk graph with ```T+x``` lines by defining a two-dimensional 
grid of "scenario" year fractions and spot prices $\tau_i \times X_j$.
Evaluate each option price (and greeks if desired) at all grid points. 
Aggregate scenario values by portfolio weights to build porfolio value as 
function of spot and year-fraction.  Subtract initial value or cost basis
to make an PnL graph.

## The Easy and the Hard

Easy
- Implementing Black-Scholes formulas for prices and greeks.
- Building a risk graph given the necessary data.

Medium
- Making sure option price data is clean.
- Implementing a robust and efficient implied volatility calculator.
- Making useful visualizations and interfaces.

Harder
- What exactly do we mean by *dte*?
- What to use for interest rates?
- What to use for forward index values?
- What to use for spot index value and dividend yield?
- What to use for option marks?  Mid or something else?

Here, we deal with the harder ones.

## Why do we care?

- Bad greeks mess up portfolio construction, hedging and performance attribution.
- Bad `T+x` lines mess up profit targets.
- Especially important when mixing call and put options.
- Implied vol skews for calls and puts may not paste together smoothly.
- If you can't analyze box-spreads correctly, what can you do?

## What do we mean by *dte*?

- SPX options have AM and PM expirations.
- Settlment timing conventions.

Fortunately, it might not matter:
 - The parameters $r, \delta, \sigma$ show up in the Black-Scholes formula only through
 $r \tau, \delta \tau, \sigma \sqrt{\tau}$.
 - If you calibrate everything to the option chain, the parameters need not 
 be interpreted too literally.
 - When tracking dollar-quantities it may not matter.
 
Sometimes it does matter:
- Tracking implied vol over time, comparing one date to another.
- PnL attribution to implied vol changes.
- Comparing implied vols between AM and PM expirations.
- Be careful with short-expirations.

Here, I use `dte = 1 + (exp_date - snap_date)` and $\tau=\text{dte}/365$.

## What to use for interest rates?

1. Interpolate 1m, 3m, 6m, 12m Libor rates.
2. Interpolate 1w-2y Sofr OIS swap rates.
3. Use a constant rate for all maturities (maybe Fed Funds Effective or Sofr)
4. Use a rate implied from Box-Spread prices.

Example:  Your $200k margin account holds 100% allocation to a T-bill ETF.
You want to buy 50% allocation to SPY.  What should you do?
- Buy $100k of SPY on margin and pay the broker 6% interest.
- Sell a 1000-point wide call and put spread with the same strikes (expiring in say 1 year)
  and use the cash to buy SPY.  Broker gets no interest.  You recieve almost $100k in cash now
  and must pay $100k cash at expiration.  The difference can be interpreted as interest.
  The rate will be close to what broker-dealers can issue floating rate bonds for.
  The specific *dte* in this setup will depend on the timing of margin interest relief 
  and settlement conventions for SPX options.

Note: the difference between an "iron" condor or butterfly and a normal one is a box spread.
Many people perfer the iron version because it maintains higher cash levels. 

In practice, a variety of box spreads prices can be averaged to extract
a discount factor for each maturity.  Combined with a *dte* convention,
this implies an interest rate for each maturity.

- Box spreads for short maturities give poor interest rate estimates.
- Narrow box spreads give poor estimates.
- Wide OTM bid/ask spreads limit precision of estimates.

## What to use for forward index values?

For European index options, no-arbitrage implies the Put-Call Parity relation:
$$
C - P = D(F - K).
$$

- Differencing this equation between strikes makes the $DF$ term drop out.  
This how we extract discount factors from box spread prices:
$$
(C_1 - P_1) - (C_2 - P_2) = D(K_2 - K_1).
$$
- Given discount factors for each maturity, forward prices can be estimated by solving
for $F$ and averaging over a strike range with tight bid-ask spreads:
$$
F = D^{-1}(C_K - P_K) + K.
$$

## What to use for spot index and dividend yield?

- If you observe the spot index, $S$, along with the option chain, can use
$F = S \exp((r-\delta)\tau)$ to solve for dividend yield, $\delta$.
- OptionMetrics does something similar to this using end of day data over several
months to estimate a dividend yield.
- If you don't know anything (like me) you might use the forward price on the 
nearest maturity as a proxy for spot.  Then solve for dividend yield as above.

## What to use for option marks?

- Most use mid prices.  OK for ITM options, but deep OTM bid/ask spreads 
can be very wide. 
- Wide OTM bid/ask spreads introduce uncertainty into box-spread prices,
making it harder to estimate interest rates.
- Deep OTM mid prices may be inconsistent with any implied vol.
- One solution is to use mid prices for OTM options and let Put-Call parity 
determine ITM option marks.  

## Solving for everything at once

We can use convex optimization technology to help.  

Pick a maturity and define:
- $m^C_K, m^P_K$ are the mid prices for calls and puts.
- $s^C_K, s^P_K$ are the bid/ask spreads for calls and puts.
- $\epsilon^C_K, \epsilon^P_K$ determines where inside the bid/ask spread we choose our mark.
Mid price corresponds to 0, bid to -1/2 and ask to 1/2.
- $D$ is the discount factor.
- $U = DF$ is the ex-dividend spot price.

The idea is to pick the discount factor, $D$, and ex-dividend spot price, $U$, as well 
as option mark locations, $\epsilon^C_K, \epsilon^P_K$, to minimize 
the distance between the mark and mid prices:
$$
\sum_K | \epsilon^C_K | + | \epsilon^P_K |
$$
subject to the constraints that put-call parity holds at each strike,
$$
C_K - P_K = U - DK,
$$
where the marks are $C_K = m^C_K + s^C_K \epsilon^C_K$ and $P_K = m^P_K + s^P_K \epsilon^P_K$,
and they lie inside the bid/ask spread,
$$
-1/2 \leq \epsilon^{C,P}_K \leq 1/2.
$$